# Deep Autoencoding Gaussian Mixture Model for Unsupervised Anomaly Detection

In [ ]:
import numpy as np 
import pandas as pd
from data_loader import *
from main import *

## CIC IDS2017

In [2]:
class hyperparams():
    def __init__(self, config):
        self.__dict__.update(**config)
defaults = {
    'lr' : 1e-4,
    'num_epochs' : 2,
    'batch_size' : 1024,
    'gmm_k' : 4,
    'lambda_energy' : 0.1,
    'lambda_cov_diag' : 0.005,
    'pretrained_model' : None,
    'mode' : 'train',
    'use_tensorboard' : False,
    'data_path' : './data',
    'ds': 'cicids2017',

    'log_path' : './dagmm/logs',
    'model_save_path' : './dagmm/models',
    'sample_path' : './dagmm/samples',
    'test_sample_path' : './dagmm/test_samples',
    'result_path' : './dagmm/results',

    'log_step' : None,
    'sample_step' : 194,
    'model_save_step' : 194,
}

In [3]:
solver = main(hyperparams(defaults))
all_scores = solver.test(energy_only=True)

DaGMM
DaGMM(
  (encoder): Sequential(
    (0): Linear(in_features=77, out_features=60, bias=True)
    (1): Tanh()
    (2): Linear(in_features=60, out_features=30, bias=True)
    (3): Tanh()
    (4): Linear(in_features=30, out_features=10, bias=True)
    (5): Tanh()
    (6): Linear(in_features=10, out_features=1, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=1, out_features=10, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=30, bias=True)
    (3): Tanh()
    (4): Linear(in_features=30, out_features=60, bias=True)
    (5): Tanh()
    (6): Linear(in_features=60, out_features=77, bias=True)
  )
  (estimation): Sequential(
    (0): Linear(in_features=3, out_features=10, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=10, out_features=4, bias=True)
    (4): Softmax(dim=1)
  )
)
The number of parameters: 13822


100%|██████████| 518/518 [00:09<00:00, 52.13it/s]


======================TEST MODE======================
N: 529481
phi :
 tensor([0.2752, 0.1372, 0.3324, 0.2552], device='cuda:0',
       grad_fn=<DivBackward0>)
mu :
 tensor([[ 0.2248,  1.6692, -0.2704],
        [ 0.3192,  1.5797, -0.1960],
        [ 0.2104,  1.6924, -0.2851],
        [ 0.2346,  1.6660, -0.2648]], device='cuda:0', grad_fn=<DivBackward0>)
cov :
 tensor([[[ 0.3950, -0.4448,  0.3357],
         [-0.4448,  0.5862, -0.4077],
         [ 0.3357, -0.4077,  0.2957]],

        [[ 0.4153, -0.4488,  0.3465],
         [-0.4488,  0.5682, -0.4034],
         [ 0.3465, -0.4034,  0.2991]],

        [[ 0.3971, -0.4506,  0.3387],
         [-0.4506,  0.5936, -0.4130],
         [ 0.3387, -0.4130,  0.2989]],

        [[ 0.4022, -0.4513,  0.3413],
         [-0.4513,  0.5894, -0.4118],
         [ 0.3413, -0.4118,  0.2997]]], device='cuda:0',
       grad_fn=<DivBackward0>)


In [7]:
label_names = ['BENIGN','FTP-Patator','SSH-Patator','DoS slowloris','DoS Slowhttptest','DoS Hulk','DoS GoldenEye',
          'Heartbleed','Web Attack','Infiltration', 'Bot', 'PortScan', 'DDoS']

In [9]:
y_test = solver.data_loader.dataset.real_test_labels

In [10]:
fpr = 0.01
benign_scores = all_scores[y_test=='BENIGN']
benign_scores_sorted = np.sort(benign_scores)
thr_ind = int(np.ceil(len(benign_scores_sorted)*fpr))
thr = benign_scores_sorted[-thr_ind]
print (thr)

6.719545


In [11]:
for i in range(len(label_names)):
    #### Exclude web attacks from results
    if label_names[i].find('Web')>=0:
        continue
    scores = all_scores[y_test==label_names[i]]
    if i==0:
        fpr = "{0:0.4f}".format(np.sum(scores>=thr)/(0. + len(scores)))
        print('FPR:',fpr)
    else:
        tpr = "{0:0.4f}".format(np.sum(scores>=thr)/(0. + len(scores)))
        print(label_names[i]+':',tpr)

FPR: 0.0100
FTP-Patator: 0.0103
SSH-Patator: 0.0097
DoS slowloris: nan
DoS Slowhttptest: nan
DoS Hulk: nan
DoS GoldenEye: nan
Heartbleed: nan
Infiltration: 0.0278
Bot: nan
PortScan: nan
DDoS: nan


/tmp/ipykernel_19716/2539190723.py:10: RuntimeWarning: invalid value encountered in divide
  tpr = "{0:0.4f}".format(np.sum(scores>=thr)/(0. + len(scores)))
